# EDA: SoundCloud Catalog (2012-2018)

- **D1) Coverage & Quality**: Row counts, null rates, playback_count distribution
- **D2) Text Diagnostics**: Length distributions, URL frequency, top genres/tags
- **D3) Mixed-Audio Leakage**: Episode/podcast pattern detection

In [1]:
import sys
sys.path.insert(0, "..")

import polars as pl
from pathlib import Path
from config.settings import PROCESSED_DIR, YEAR_RANGE

## Load Data

In [2]:
dfs = {}
for year in YEAR_RANGE:
    year_dir = PROCESSED_DIR / f"year={year}"
    if year_dir.exists():
        parquet_files = list(year_dir.glob("*.parquet"))
        if parquet_files:
            dfs[year] = pl.concat([pl.read_parquet(f) for f in parquet_files])
            print(f"{year}: {dfs[year].height:,} rows")

2012: 6,209,364 rows
2013: 7,955,469 rows
2014: 6,546,780 rows
2015: 6,014,630 rows
2016: 5,227,711 rows
2017: 4,216,928 rows
2018: 4,657,187 rows


In [3]:
df_all = pl.concat(list(dfs.values()))
print(f"Total rows: {df_all.height:,}")
print(f"Columns: {df_all.columns}")

Total rows: 40,828,069
Columns: ['title', 'artist', 'year', 'created_at', 'genre', 'tags', 'description', 'playback_count', 'permalink_url', 'extracted_vibe_text', 'ingest_date', 'track_id']


## D1) Coverage & Quality
### Row Count Per Year

In [4]:
year_counts = [(year, dfs[year].height) for year in sorted(dfs.keys())]
for year, count in year_counts:
    print(f"{year}: {count:,}")

2012: 6,209,364
2013: 7,955,469
2014: 6,546,780
2015: 6,014,630
2016: 5,227,711
2017: 4,216,928
2018: 4,657,187


### Null/Empty Rates Per Field

In [5]:
null_stats = []
for col in df_all.columns:
    null_count = df_all[col].null_count()
    null_pct = (null_count / df_all.height) * 100
    if df_all[col].dtype == pl.Utf8:
        empty_count = df_all.filter(pl.col(col) == "").height
        empty_pct = (empty_count / df_all.height) * 100
    else:
        empty_count = 0
        empty_pct = 0.0
    null_stats.append({"column": col, "null_pct": round(null_pct, 2), "empty_pct": round(empty_pct, 2)})

null_df = pl.DataFrame(null_stats)
print(null_df)

shape: (12, 3)
┌─────────────────────┬──────────┬───────────┐
│ column              ┆ null_pct ┆ empty_pct │
│ ---                 ┆ ---      ┆ ---       │
│ str                 ┆ f64      ┆ f64       │
╞═════════════════════╪══════════╪═══════════╡
│ title               ┆ 0.95     ┆ 0.0       │
│ artist              ┆ 10.26    ┆ 0.0       │
│ year                ┆ 0.0      ┆ 0.0       │
│ created_at          ┆ 0.0      ┆ 0.0       │
│ genre               ┆ 37.95    ┆ 0.0       │
│ …                   ┆ …        ┆ …         │
│ playback_count      ┆ 0.62     ┆ 0.0       │
│ permalink_url       ┆ 0.0      ┆ 0.0       │
│ extracted_vibe_text ┆ 0.49     ┆ 0.0       │
│ ingest_date         ┆ 0.0      ┆ 0.0       │
│ track_id            ┆ 0.0      ┆ 0.0       │
└─────────────────────┴──────────┴───────────┘


### Playback Count Distribution

In [6]:
if "playback_count" in df_all.columns:
    pc = df_all["playback_count"].drop_nulls()
    print(f"Min: {pc.min()}")
    print(f"Max: {pc.max():,}")
    print(f"Mean: {pc.mean():,.0f}")
    print(f"Median: {pc.median():,.0f}")
    print(f"Zero count: {(pc == 0).sum():,} ({(pc == 0).sum() / len(pc) * 100:.1f}%)")

Min: 0
Max: 328,034,491
Mean: 111,401
Median: 0
Zero count: 29,149,376 (71.8%)


## D2) Text Diagnostics
### Description Length Distribution

In [7]:
if "description" in df_all.columns:
    desc_lens = df_all["description"].str.len_chars().drop_nulls()
    print(f"Min: {desc_lens.min()}")
    print(f"Max: {desc_lens.max():,}")
    print(f"Mean: {desc_lens.mean():,.0f}")
    print(f"Median: {desc_lens.median():,.0f}")
    print(f"Empty (len=0): {(desc_lens == 0).sum():,}")

Min: 1
Max: 64,220
Mean: 397
Median: 181
Empty (len=0): 0


### Vibe Text Length Distribution

In [8]:
if "extracted_vibe_text" in df_all.columns:
    vibe_lens = df_all["extracted_vibe_text"].str.len_chars().drop_nulls()
    print(f"Min: {vibe_lens.min()}")
    print(f"Max: {vibe_lens.max():,}")
    print(f"Mean: {vibe_lens.mean():,.0f}")
    print(f"Median: {vibe_lens.median():,.0f}")

Min: 1
Max: 64,426
Mean: 130
Median: 39


### URL Frequency in Description (Spam Proxy)

In [9]:
if "description" in df_all.columns:
    url_pattern = r"https?://"
    has_url = df_all["description"].str.contains(url_pattern).sum()
    print(f"Descriptions with URLs: {has_url:,} ({has_url / df_all.height * 100:.1f}%)")

Descriptions with URLs: 0 (0.0%)


### Top Genres

In [10]:
if "genre" in df_all.columns:
    genre_counts = df_all.group_by("genre").len().sort("len", descending=True).head(20)
    print(genre_counts)

shape: (20, 2)
┌────────────────────────┬──────────┐
│ genre                  ┆ len      │
│ ---                    ┆ ---      │
│ str                    ┆ u32      │
╞════════════════════════╪══════════╡
│ null                   ┆ 15495148 │
│ Electronic             ┆ 1120651  │
│ Hip hop Rap            ┆ 873783   │
│ House                  ┆ 569017   │
│ Techno                 ┆ 555621   │
│ …                      ┆ …        │
│ Dance EDM              ┆ 233384   │
│ Indie                  ┆ 225212   │
│ Alternative            ┆ 210337   │
│ Folk Singer Songwriter ┆ 204779   │
│ Entertainment          ┆ 197535   │
└────────────────────────┴──────────┘


### Top Tags

In [11]:
if "tags" in df_all.columns:
    tag_counts = df_all.group_by("tags").len().sort("len", descending=True).head(20)
    print(tag_counts)

shape: (20, 2)
┌─────────────────────────────────┬──────────┐
│ tags                            ┆ len      │
│ ---                             ┆ ---      │
│ str                             ┆ u32      │
╞═════════════════════════════════╪══════════╡
│ null                            ┆ 34265256 │
│ soundcloud source iphone recor… ┆ 26474    │
│ soundcloud source android 3rdp… ┆ 26253    │
│ Podcast                         ┆ 16820    │
│ soundcloud source android reco… ┆ 14789    │
│ …                               ┆ …        │
│ House                           ┆ 3725     │
│ podcast                         ┆ 3618     │
│ Speech                          ┆ 2690     │
│ Soundtrack                      ┆ 2646     │
│ Cox Crendor Comedy Podcast New… ┆ 2522     │
└─────────────────────────────────┴──────────┘


## D3) Mixed-Audio Leakage Detection
### Episode/Podcast Patterns in Titles

In [12]:
patterns = {
    "ep/episode + number": r"\b(ep|episode)\s*[\.:#-]?\s*\d+",
    "season+episode": r"\bs\s*\d+\s*e\s*\d+",
    "podcast": r"\bpodcast\b",
    "interview": r"\binterview\b",
    "with guest": r"\bwith\s+guest\b",
    "full album": r"\bfull\s+album\b",
    "mix": r"\bmix\b",
    "radio": r"\bradio\b",
}

if "title" in df_all.columns:
    title_lower = df_all.with_columns(pl.col("title").str.to_lowercase().alias("title_l"))
    for name, pattern in patterns.items():
        count = title_lower["title_l"].str.contains(pattern).sum()
        pct = count / df_all.height * 100
        print(f"{name}: {count:,} ({pct:.2f}%)")

ep/episode + number: 578,610 (1.42%)
season+episode: 29,261 (0.07%)
podcast: 430,152 (1.05%)
interview: 134,958 (0.33%)
with guest: 3,416 (0.01%)
full album: 6,909 (0.02%)
mix: 2,433,535 (5.96%)
radio: 537,102 (1.32%)


### Blocked Genres Check

In [13]:
BLOCK_GENRES = {"sports", "spoken words", "spoken word"}

if "genre" in df_all.columns:
    genre_lower = df_all.with_columns(pl.col("genre").str.to_lowercase().alias("genre_l"))
    blocked = genre_lower.filter(pl.col("genre_l").is_in(BLOCK_GENRES))
    print(f"Blocked genres total: {blocked.height:,} ({blocked.height / df_all.height * 100:.2f}%)")
    print(blocked.group_by("genre_l").len().sort("len", descending=True))

Blocked genres total: 122,896 (0.30%)
shape: (3, 2)
┌──────────────┬────────┐
│ genre_l      ┆ len    │
│ ---          ┆ ---    │
│ str          ┆ u32    │
╞══════════════╪════════╡
│ sports       ┆ 106799 │
│ spoken word  ┆ 15690  │
│ spoken words ┆ 407    │
└──────────────┴────────┘


## Summary: Recommended Thresholds

In [14]:
print("Based on EDA, recommended values:")
print("- BLOCK_GENRES: sports, spoken words, spoken word")
print("- MAX_DESC_LENGTH: 400 chars")
print("- Episode/podcast regex patterns confirmed above")

Based on EDA, recommended values:
- BLOCK_GENRES: sports, spoken words, spoken word
- MAX_DESC_LENGTH: 400 chars
- Episode/podcast regex patterns confirmed above
